In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
## 武汉
# roi = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
# Map.addLayer(roi, {}, "roi")
# Map.centerObject(roi,8)

# 青海省部分地区
roi = ee.Geometry.Rectangle([89.852623, 34.537942,92.3677, 36.217184])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [19]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
# landsat影像可视化参数
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)

def area_after_clip(image):
    image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    water_area = image.clip(roi).select('mNDWI').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14
    })
    waterareas = water_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14
    })    
    return image.setMulti({'imagearea': imageareas.get('B2'),'waterarea': waterareas.get('mNDWI')})

In [20]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(water_index)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')
print(landsat_images.size().getInfo())

154


# 筛选得出采样影像

In [21]:
image = landsat_images.mosaic().clip(roi)
image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14
})
print(areas.getInfo())

# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('imagearea',ee.Number(areas.get('B2')).divide(ee.Number(10))))
dates = sample_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

{'B2': 41728.58360534594}


In [28]:
water_area = landsat_images.map(area_after_clip).filter(ee.Filter.gt('waterarea',5)).aggregate_array('waterarea')
print(water_area.size().getInfo())
print(water_area.getInfo())

water_area = landsat_images.map(area_after_clip).filter(ee.Filter.lte('waterarea',5)).aggregate_array('waterarea')
print(water_area.size().getInfo())
print(water_area.getInfo())

123
[6.318239109068627, 431.84146253774526, 169.42831107696063, 376.7957919122545, 224.86056090637274, 356.56428758063686, 271.86227782107824, 2101.5151205779357, 272.4478725534318, 276.93434414681366, 597.3186836249992, 184.41271114044127, 497.3936333806376, 154.75347806617643, 698.3107105625007, 224.85637859338223, 178.901523164706, 1975.009216815435, 1078.3783154889677, 1833.0586279576034, 1079.0136242578437, 416.51667986078434, 462.24820135392144, 351.61231275049016, 131.63101704289215, 247.60283292107863, 232.88334739877465, 211.55947887573546, 181.98254718210774, 237.81171710735305, 396.83912538284295, 168.02325681249997, 394.6510496568629, 253.52034362573525, 133.69168888014707, 456.4569913615199, 593.3370703754903, 185.90985256127453, 179.46488798970574, 1239.146419574999, 1844.5973543495168, 828.9167334340689, 570.6729246838236, 172.73363473333345, 737.0479099823519, 631.1349784495096, 161.11274469607844, 795.6861254779407, 511.0807988723037, 1608.744612953683, 485.89328825367

# 样本采集函数的定义

In [30]:
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
Map.addLayer(index_waterfrequency.gte(0.75).selfMask(),{'palette':['green']},"index permanentwater")
# JRC
JRC_2018 = ee.Image("JRC/GSW1_3/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# # Maryland
# Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').rename('waterclass')
# Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
# Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
# Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# # 三者的交集
# permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
# landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
# seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
# 两者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

In [34]:
def intervalmean(image):
    awei = image.clip(roi).updateMask(seasonWaterExtent).select('AWEI')
    intervalMean = awei.reduceRegion(**{
        'reducer': ee.Reducer.intervalMean(50,100), 
        'geometry': roi, 
        'scale': 30, 
        'bestEffort': True
    })
    return image.set({'mean': intervalMean.get('AWEI')})

In [43]:
a = sample_image.limit(60).map(intervalmean)
b = a.aggregate_array('mean')
print(b.getInfo())

[512.0451213139038, -114.49796079370304, 756.9345351922501, 1302.979799103321, 792.7658161906551, 2798.9744252484443, 143.93344343505905, 1349.6538123304447, 1513.1129256320016, 1381.2949121190823, 1454.3913046842986, 1931.4539373878463, 1390.7043063582657, 2219.310380322445, 378.55925787884286, 717.1790325480788, 348.5601737235757, 426.4619569553601, 328.37584428905075, 84.66627420510734, 561.6733373055937, 829.5404368775124, 137.73620226778533, 835.9416580166903, 1017.2339477892169, 734.2716284353187, 1215.309907946221, 1460.9975797543318, 1254.1859176747253, 1396.0357053553457, 1481.0839552470356, 156.2040514118646, 630.4515060354635, 320.0221648872676, 1695.7381950792926, 96.45512622689097, -246.96116583555403, 2102.7320549184856, 7.872872008172088, 163.4939949486067, -671.72598919444, 1359.5643083137572, 1636.7724021042502, 1906.9196151861272, 2583.4091465706874, 1844.2416762296407, 148.96127306065654, 176.4132912082515, 1125.281730271421, 1798.2991903737513, 852.5745425372392, 24

In [ ]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi','AWEI']
# 分类标签
label = 'waterclass'
def imageSample(image):
    ROI = image.clip(roi).geometry()
    permanent = permanentWaterExtent.updateMask(image.select('B3').gt(0)).remap([0,1],[0,2]).rename('waterclass')
    land = landExtent.updateMask(image.select('B3').gt(0))
    seasonalwater_Extent = image.updateMask(seasonWaterExtent).select('AWEI').gt(ee.Image.constant(image.get('mean'))).remap([0,1],[0,2]).rename('waterclass')
    percentile_waterclass = ee.ImageCollection([permanent,seasonalwater_Extent,land]).sum().remap([0,1,2],[0,0,1]).rename('waterclass')
    nowater = percentile_waterclass.remap([0,1],[1,0]).rename('waterclass').selfMask()
    water_points = percentile_waterclass.selfMask().sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 200,
        'seed': 1,
        'geometries': True,
    })
    nowater_points = nowater.sample(**{
        'region': ROI,
        'scale': 30,
        'numPixels': 50,
        'seed': 1,
        'geometries': True,
    })
    all_points = water_points.merge(nowater_points)
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30,
        'geometries': True,
    })
    return sample_points

In [ ]:
# 总的样本点数
points_collection = a.map(imageSample).flatten()
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))

# # 水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

In [ ]:
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

In [ ]:
# 添加一个底图白板
# image9 = water_index(maskL8sr(ee.Image(landsat_images.toList(landsat_images.size()).get(5))))
image9 = water_index(maskL8sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20180627')))
Map.addLayer(image9.clip(roi),visParams,'image9')

basemap = ee.Image.constant(0).toFloat().clip(image9.geometry()).clip(roi).rename('waterclass')
Map.addLayer(basemap,{'palette':['white']},"basemap")

In [ ]:
classify_image = image9.clip(roi).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
# Map.addLayer(classify_image.selfMask(),{'palette':['green']}, 'classify image')
Map.addLayer(classify_image,{'palette':['red','blue'],'min':0,'max':1}, 'classify image')

In [ ]:
def AutomaticCorrection(image):
    basemap = ee.Image.constant(0).toFloat().clip(image.geometry()).clip(roi).rename('waterclass')
    class_image = ee.ImageCollection([basemap,image.remap([0,1],[1,2]).rename('waterclass')]).sum()
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence').clip(image.geometry()).clip(roi)
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': image.clip(roi).geometry(),
        'scale': 90,
    })        
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    print(occurrence_threshold.getInfo())
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

In [ ]:
a = AutomaticCorrection(classify_image)
Map.addLayer(a,{'palette':['white','red'],'min':0,'max':1}, 'enhanced_water')

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
raw_image = maskL8sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20180627').clip(roi))
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(raw_image, region=region, vis_params=visParams)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'reference image(masked)', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
vis = {'palette':['D4D4D4','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(classify_image, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'classified image', fontsize=20)